<a href="https://colab.research.google.com/github/hardiksiloiya/GANs/blob/main/Simple_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Activation,LSTM,GRU,Dense,Embedding,Input
import glob
from keras.models import Model
import yaml
from keras import preprocessing,utils
import numpy as np
from tensorflow.keras.utils import to_categorical


In [2]:
!wget https://github.com/shubham0204/Dataset_Archives/blob/master/chatbot_nlp.zip?raw=true -O chatbot_nlp.zip
!unzip chatbot_nlp.zip


--2021-06-23 08:38:22--  https://github.com/shubham0204/Dataset_Archives/blob/master/chatbot_nlp.zip?raw=true
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/shubham0204/Dataset_Archives/raw/master/chatbot_nlp.zip [following]
--2021-06-23 08:38:22--  https://github.com/shubham0204/Dataset_Archives/raw/master/chatbot_nlp.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shubham0204/Dataset_Archives/master/chatbot_nlp.zip [following]
--2021-06-23 08:38:22--  https://raw.githubusercontent.com/shubham0204/Dataset_Archives/master/chatbot_nlp.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:

In [3]:
questions = list()
answers = list()

for file in glob.glob('chatbot_nlp/data/*'):
    stream = open(file,'rb')
    info=yaml.safe_load(stream)
    conversations=info['conversations']
    for con in conversations:
        if len(con)>2 :
            questions.append(con[0])
            replies=con[1:]
            ans=''
            for rep in replies:
                ans+=' ' + rep
            answers.append(ans)
        elif len(con)> 1:
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = list()
for i in range(len(answers)):
    if type(answers[i]) == str:
        answers_with_tags.append(answers[i])
    else:
        questions.pop(i)

answers = list()
for i in range(len(answers_with_tags)) :
    answers.append('<start> '+answers_with_tags[i]+' <end>')

tokenizer=preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions+answers)
size=len( tokenizer.word_index )+1

In [5]:
def tokenize(sentences):
    token_list=[]
    vocabu=[]
    for sentence in sentences:
        sentence=sentence.lower()
        sentence=re.sub('[^a-zA-Z]',' ',sentence)
        tokens=sentence.split()
        vocabu+=tokens
        token_list.append(tokens)
    return token_list,vocabu

In [6]:
from gensim.models import Word2Vec
import re

vocab = []
for word in tokenizer.word_index:
    vocab.append( word )

token_questions=tokenizer.texts_to_sequences(questions)
m=max([len(x) for x in token_questions])
pad_questions=preprocessing.sequence.pad_sequences(token_questions,maxlen=m ,padding='post')
encoder_input_data=np.array(pad_questions)
print(encoder_input_data.shape,m)

token_answers=tokenizer.texts_to_sequences(answers)
m2=max([len(x) for x in token_answers])
pad_answers=preprocessing.sequence.pad_sequences(token_answers,maxlen=m2,padding='post')
decoder_input_data=np.array(pad_answers)
print(decoder_input_data.shape,m2)

token_answers=tokenizer.texts_to_sequences(answers)
for i in range(len(token_answers)):
    token_answers[i]=token_answers[i][1:]
pad_answers=preprocessing.sequence.pad_sequences(token_answers,maxlen=m2,padding='post')
coding=to_categorical(pad_answers,size)
decoder_output_data=np.array(coding)
print(decoder_output_data.shape)

(564, 22) 22
(564, 74) 74
(564, 74, 1894)


In [7]:
encoder_inputs=Input(shape=(m,))
encoder_embedding=Embedding(size,200,mask_zero=True)(encoder_inputs)
encoder_outputs,state_h,state_c=LSTM(200,return_state=True)(encoder_embedding)
encoder_states=[state_h,state_c]

decoder_inputs=Input(shape=(m2,))
decoder_embedding=Embedding(size,200,mask_zero=True)(decoder_inputs)
decoder_lstm=LSTM(200,return_state=True,return_sequences=True)
decoder_outputs,t1,t2=decoder_lstm(decoder_embedding,initial_state=encoder_states)
decoder_dense=Dense(size,activation=tf.keras.activations.softmax) 
output=decoder_dense( decoder_outputs )

model=Model([encoder_inputs,decoder_inputs],output)
model.compile(optimizer=tf.keras.optimizers.RMSprop(),loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 22)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 74)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 22, 200)      378800      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 74, 200)      378800      input_2[0][0]                    
______________________________________________________________________________________________

In [8]:
model.fit([encoder_input_data,decoder_input_data],decoder_output_data,batch_size=50,epochs=150) 
model.save('temp.h5') 

Epoch 1/150
12/12 [==============================] - 23s 35ms/step - loss: 1.4284
Epoch 2/150
12/12 [==============================] - 0s 34ms/step - loss: 1.1906
Epoch 3/150
12/12 [==============================] - 0s 34ms/step - loss: 1.1016
Epoch 4/150
12/12 [==============================] - 0s 35ms/step - loss: 1.0321
Epoch 5/150
12/12 [==============================] - 0s 35ms/step - loss: 1.0368
Epoch 6/150
12/12 [==============================] - 0s 34ms/step - loss: 1.0293
Epoch 7/150
12/12 [==============================] - 0s 35ms/step - loss: 1.0726
Epoch 8/150
12/12 [==============================] - 0s 34ms/step - loss: 1.0399
Epoch 9/150
12/12 [==============================] - 0s 35ms/step - loss: 1.0504
Epoch 10/150
12/12 [==============================] - 0s 34ms/step - loss: 0.9937
Epoch 11/150
12/12 [==============================] - 0s 35ms/step - loss: 0.9767
Epoch 12/150
12/12 [==============================] - 0s 34ms/step - loss: 0.9628
Epoch 13/150
12/12 [====

In [12]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h=Input(shape=(200,))
    decoder_state_input_c=Input(shape=(200,))
    
    decoder_states_inputs=[decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs,state_h,state_c=decoder_lstm(decoder_embedding,initial_state=decoder_states_inputs)
    decoder_states=[state_h,state_c]
    decoder_outputs=decoder_dense(decoder_outputs)
    decoder_model=Model([decoder_inputs]+decoder_states_inputs,[decoder_outputs] + decoder_states)
    
    return encoder_model,decoder_model

In [15]:
def str_to_tokens( sentence : str ):
    words=sentence.lower().split()
    tokens_list=list()
    for word in words:
        tokens_list.append(tokenizer.word_index[word]) 
    return preprocessing.sequence.pad_sequences([tokens_list],maxlen=m,padding='post')


In [25]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values=enc_model.predict(str_to_tokens(input('Enter question : ')))
    empty_target_seq=np.zeros((1,1))
    empty_target_seq[0, 0]=tokenizer.word_index['start']
    tt=False
    final=''
    while not tt :
        dec_outputs,h,c=dec_model.predict([empty_target_seq]+states_values)
        t_ind=np.argmax(dec_outputs[0,-1,:])
        temp=None
        for word,index in tokenizer.word_index.items() :
            if t_ind == index :
                final+=' {}'.format(word)
                temp=word
        
        if temp=='end' or len(final.split()) > m:
            tt=True
            
        empty_target_seq=np.zeros((1,1))  
        empty_target_seq[0,0]=t_ind
        states_values=[h,c] 

    print(final)

Enter question : hi
 hello end
Enter question : how are you
 i have no i'm a piece of software end
Enter question : where are you
 i am on the internet end
Enter question : why
 i support the 2nd amendment end
Enter question : are you okay
 no i am sober nope not noticeably i'm software i'm not so i can be programmed to act and react as if i
Enter question : do you talk
 no i am not into sports that myself end
Enter question : do you like me
 what is basketball i frighten like that much myself end
Enter question : quit


KeyError: ignored